In [ ]:
import bson 
import json
import numpy as np
import pandas as pd
import time
import sys
import scipy.sparse
import pickle
import sys
import re
from tqdm import tqdm_notebook as tqdm
import time
from mtranslate import translate
from langdetect import detect
from langdetect import detect_langs
from gensim.models import word2vec

In [ ]:
f2 = open("all_top_skills_final_key.txt","rb")
key_list = pickle.load(f2)
f2.close()
f2 = open("all_top_skills_final_fre.txt","rb")
fre_save = pickle.load(f2)
f2.close()
f= file('word2vec_model_allskills', 'rb')
word2vec=pickle.load(f)

In [ ]:
f2 = open("document_per_infor.txt","rb")
document = pickle.load(f2)
f2.close()

In [ ]:
skill_titlt = []
for i in tqdm(range(len(data_profile))):
#for i in tqdm(range(5000,10000)):
    if 'skills' in (data_profile[i].keys()):
        for skills_divided in (data_profile[i]['skills']): 
            ## data_profile[0]['skills']是个列表，所以skills_divided就代表了topskill等的字典
             #for skills_detail in skills_divided['skills']:  ##skills_divided['skills']又是列表
                if skills_divided['title'] not in skill_titlt:
                    skill_titlt.append(skills_divided['title'])
print skill_titlt

cata_title = []
for i in tqdm(range(len(data_profile))):
    for x in data_profile[i].keys():
        if x  not in cata_title:
            cata_title.append(x)
print cata_title

In [ ]:
title = []
for i in tqdm(range(len(data_profile))):
    if 'personalInformation' in data_profile[i].keys():
        for term in data_profile[i]['personalInformation'].keys():
            if term not in title:
                title.append(term)
print title

title = []
for i in tqdm(range(len(data_profile))):
    if 'educations' in data_profile[i].keys():
        for term in data_profile[i]['educations']:
             for name in term.keys():
                if name not in title:
                    title.append(name)

print title

title = []
for i in tqdm(range(len(data_profile))):
    if 'experiences' in data_profile[i].keys():
        for term in data_profile[i]['experiences']:
             for name in term.keys():
                if name not in title:
                    title.append(name)

print title

In [ ]:
information = []
per_infor_chosen = [u'currentJob',u'headline',u'currentJobCompany']
for i in tqdm(range(len(data_profile))):
    if u'personalInformation' in data_profile[i].keys():
        information.append('personalInformation')  
        for term in data_profile[i]['personalInformation'].keys():
            if term in per_infor_chosen:
                try:
                    lang = detect(data_profile[i]['personalInformation'][term])
                except:
                    lang = u'fr'
                information[i] += ' '+ term
                if lang != u'en':
                    try:
                        information[i] += ' '+ translate(data_profile[i]['personalInformation'][term].encode('utf-8'))
                    except:
                        information[i] += ' '+ data_profile[i]['personalInformation'][term]
                else:
                    information[i] += ' ' + data_profile[i]['personalInformation'][term]
    else:
        information.append('')  

In [ ]:
accomplishments=[]
accompl_chosen_1 = [ u'projets', u'Projects', u'Project', u'projet']
#accompl_chosen_2 = [u'publication', u'Publications',u'publications', u'Publication']
for i in range(len(data_profile)):
    if u'accomplishments' in data_profile[i].keys():
        #########################################accomplishments is not a dictionnary!
        for term in data_profile[i]['accomplishments']:
            if term['title'] in accompl_chosen_1:
                accomplishments.append('project')
                ######################entry
                for _term in term:
                    if u'description' in _term.keys():
                        accomplishments[i] += ' '+ 'description'
                        try:
                            lang = detect(_term['description'])
                        except: 
                            lang = u'fr'
                        if lang != u'en':
                            try:
                                accomplishments[i] += ' '+ translate(_term['description'].encode('utf-8'))
                            except:
                                accomplishments[i] += ' '+ _term['description']
                        else:
                            accomplishments[i] += ' ' +  _term['description']
    else:
        accomplishments.append('')   

In [ ]:
educations = []
edu_chosen = [u'Description', u'fieldOfStudy', u'ActivitiesAndSocieties']
for i in tqdm(range(len(data_profile))):
    if u'educations' in data_profile[i].keys():
        education.append('educations')
        for term in data_profile[i]['education']:
            for title in edu_chosen:
                if title in term.keys():
                        try:
                            lang = detect(term[title])
                        except: 
                            lang == u'fr'
                        educations[i] += ' '+ title
                        if lang != u'en':
                            try:
                                educations[i] += ' '+ translate(term[title].encode('utf-8'))
                            except:
                                educations[i] += ' '+ term[title]
                        else:
                            educations[i] += ' ' +  term[title]
    else:
        educations.append('')    

In [ ]:
experiences = []
title_chosen = [ u'description',u'title',u'Company']
#for i in tqdm(range(len(data_profile))):
for i in tqdm(range(100)):
    if u'experiences' in data_profile[i].keys():
        experiences.append('experiences')
        for term in data_profile[i]['experiences']:
            for title in title_chosen:
                if title in term.keys():
                        try:
                            lang = detect(term[title])
                        except:
                            lang = u'fr'
                        experiences[i] += ' '+ title
                        if lang != u'en':
                            try:
                                experiences[i] += ' '+ translate(term[title].encode('utf-8'))
                            except:
                                experiences[i] += ' '+ term[title]
                        else:
                            experiences[i] += ' ' +  term[title]
    else:
        experiences.append('')                         

In [ ]:
# config
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

documents = []
# use count as label of each row
count = 0

for line in document:
    #title = unicode(line, 'utf-8')
    #words = ko_title2words(title)
    # 这里documents里的每个元素是二元组，具体可以查看函数文档
    documents.append(gensim.models.doc2vec.TaggedDocument(words, [str(count)]))
    count += 1
    if count % 10000 == 0:
        logging.info('{} has loaded...'.format(count))

# train
model = Doc2Vec(documents, dm=1, size=100, window=8, min_count=5, workers=4)
# save model
model.save('models/ko_d2v.model')